In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
#use tensorflow version 1.x features
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

/Users/anmol/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/anmol/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/anmol/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/anmol/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [2]:
columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

#separate feature and labels
features = columns[:-1]
label = columns[-1]

iris_train = pd.read_csv('iris_training.csv', header = None, names = columns)
iris_test = pd.read_csv('iris_test.csv', header = None, names = columns)

In [3]:
iris_train.sample(5)

,sepal_length,sepal_width,petal_length,petal_width,species
95,4.6,3.4,1.4,0.3,0
69,5.0,3.4,1.5,0.2,0
49,7.2,3.2,6.0,1.8,2
83,5.6,2.7,4.2,1.3,1
25,5.2,3.5,1.5,0.2,0


In [4]:
X_train = iris_train.iloc[:, :-1]

#one hot encode labels
y_train = iris_train.iloc[:, -1:]
y_train = pd.get_dummies(y_train.species)

X_test = iris_test.iloc[:, :-1]

#one hot encode labels
y_test = iris_test.iloc[:, -1:]
y_test = pd.get_dummies(y_test.species)

### 2. Create a simple softmax classifier with 3 neurons in output layer and no hidden layer using TensorFlow Core APIs. Save it as iris_softmax.py

Ref - 
- https://steadforce.com/first-steps-tensorflow-part-2/
- https://www.tensorflow.org/api_docs/python/tf/math/reduce_std
- https://www.kaggle.com/vijayshinva/iris-tensorflow-basic-softmax-regression


In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(120, 4)
(120, 3)
(30, 4)
(30, 3)


In [6]:
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 4])  #four features
y = tf.placeholder(tf.float32, shape=[None, 3])  #three classes

#placeholders for weights and biases
weight = tf.Variable(tf.zeros(shape=[4, 3]))
bias = tf.Variable(tf.zeros(shape=[3]))

#define softmax using tensorflow
predicted = tf.nn.softmax(tf.matmul(x, weight) + bias)

In [7]:
#define cost function
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(predicted), axis=1))

df = pd.DataFrame(columns=['Iteration','Learn Rate', 'Epoch', 'Mean Accuracy', 'Standard Deviation'])

#run whole thing 10 times
for i in range(1,11) :
    
    #try with different learn rate and iterations for gradient descent
    learn_rate = i/100
    epochs = i*1000
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learn_rate).minimize(cost)
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs) :
        sess.run([optimizer], feed_dict={x:X_train, y:y_train})

    weight_hat, bias_hat = sess.run([weight, bias])
    prediction_accuracy = tf.cast(tf.equal(tf.argmax(predicted, 1), tf.argmax(y,1)), tf.float32)
    
    mean_accuracy = sess.run(tf.reduce_mean(prediction_accuracy), feed_dict={weight:weight_hat, bias:bias_hat, 
                                                                   x: X_test, y: y_test})
    std = sess.run(tf.math.reduce_std(prediction_accuracy), feed_dict={weight:weight_hat, bias:bias_hat, 
                                                                        x: X_test, y: y_test})
    df = df.append({'Iteration': i, 'Learn Rate': learn_rate, 'Epoch': epochs,
                    'Mean Accuracy': mean_accuracy, 'Standard Deviation': std}, ignore_index=True)
 
df.head(10)

,Iteration,Learn Rate,Epoch,Mean Accuracy,Standard Deviation
0,1.0,0.01,1000.0,0.933333,0.249444
1,2.0,0.02,2000.0,0.966667,0.179505
2,3.0,0.03,3000.0,0.966667,0.179505
3,4.0,0.04,4000.0,0.966667,0.179505
4,5.0,0.05,5000.0,0.966667,0.179505
5,6.0,0.06,6000.0,0.966667,0.179505
6,7.0,0.07,7000.0,0.933333,0.249444
7,8.0,0.08,8000.0,0.933333,0.249444
8,9.0,0.09,9000.0,0.933333,0.249444
9,10.0,0.10,10000.0,0.933333,0.249444


### 3. Create an MLP classifier with 3 hidden layers of sizes 5, 10, 5 using tensorflow Core APIs. Save it as iris_mlp_tf.py

Ref :
- https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809
- https://steadforce.com/first-steps-tensorflow-part-3/

In [16]:
tf.reset_default_graph()
learning_rate = 0.01

graph = tf.Graph()

with graph.as_default() :
    x = tf.placeholder(tf.float32, [None, 4])
    y = tf.placeholder(tf.float32, [None, 3])

In [17]:
with graph.as_default() :
    hidden_layer_1 = tf.contrib.layers.fully_connected(inputs = x,
                                                      num_outputs = 5,
                                                      activation_fn=tf.nn.relu)
    hidden_layer_2 = tf.contrib.layers.fully_connected(inputs = hidden_layer_1,
                                                      num_outputs = 10,
                                                      activation_fn=tf.nn.relu)
    hidden_layer_3 = tf.contrib.layers.fully_connected(inputs = hidden_layer_2,
                                                      num_outputs = 5,
                                                      activation_fn=tf.nn.relu)
    predicted = tf.contrib.layers.fully_connected(inputs = hidden_layer_3, 
                                                  num_outputs = 3, 
                                                  activation_fn=tf.nn.softmax)
    
    cost = tf.losses.softmax_cross_entropy(y, logits=predicted)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learn_rate).minimize(cost)
    is_match = tf.equal(tf.argmax(predicted, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_match, tf.float32))
    sess.run([optimizer, cost], feed_dict={x: X_test, y: y_test})

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(?, 4), dtype=float32) is not an element of this graph.